In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import eli5
from eli5.sklearn import PermutationImportance

import tensorflow as tf
from tensorflow import keras


from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [ ]:
def filter_years(df):
    """
    filters years 2013 and 2018 which we have to handle.
    """
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'].dt.year.isin([2013, 2018])]
    return df

weather = pd.read_csv("weatherAUS.csv")  # read csv data into pandas data frame
weather = filter_years(weather)
weather['Date'].dt.year.value_counts()

# Aufgabe 1 Feature Engineering

In [ ]:
# Drop columns with missing target Variable "RainTomorrow"
weather = weather[weather['RainTomorrow'].notna()]

In [ ]:
# Test showed that dropping at the threshholds used results in slightly better results.

weather = weather[weather.columns[weather.isnull().mean() < 0.4]]
weather = weather.loc[weather.isnull().mean(axis=1) < 0.5]

In [ ]:
weather['RainToday'] = weather["RainToday"].replace({'No':0, 'Yes':1})
weather['RainTomorrow'] = weather["RainTomorrow"].replace({'No':0, 'Yes':1})

In [ ]:
weather['Year'] = weather['Date'].dt.year  # get year
weather['Month'] = weather['Date'].dt.month  # get month
weather['Day'] = weather['Date'].dt.day  # get day
weather.drop(labels=['Date', 'Location'], axis=1, inplace=True)

## Feature Binning

In [ ]:
def encode_season(month):
    if month >= 9 and month <= 11:
        return 'Spring'
    if month == 12 or month <= 2:
        return 'Summer'
    if month >= 3 and month <= 5:
        return 'Autumn'
    if month >= 6 and month <= 8:
        return 'Winter'
    
weather['Season'] = weather['Month'].apply(encode_season)

## Feate creation
creating new features

In [ ]:
weather['MinMaxDiff'] = weather['MaxTemp'] - weather['MinTemp']
weather['PressureDiff'] = weather['Pressure3pm'] - weather['Pressure9am']
weather['WindSpeedDiff'] = weather['Pressure3pm'] - weather['WindSpeed9am']
weather['HumidityDiff'] = weather['Humidity3pm'] - weather['Humidity9am']

## Train Test split
**Important!** 

Before starting Feature Engineering one must split the dataset to ovoid test train leakage!
All Decisions in Data Engineering must be made on the Train Set only! From here, we assume that we dont have any 
knowledge about the test data.

In [ ]:
train, test = train_test_split(weather, test_size=0.2, random_state = 0)

## Outlier detection
**important**

this must be done after splitting. Again: We do not know anything about the test data!

-> Hat noch nix gebracht.

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(2, 2, 1)
fig = train.boxplot(column='Rainfall')
fig.set_title('')
fig.set_ylabel('Rainfall')

plt.subplot(2, 2, 2)
fig = train.boxplot(column='WindGustSpeed')
fig.set_title('')
fig.set_ylabel('WindGustSpeed')

In [ ]:
higher_lim = train['Rainfall'].quantile(0.995)

train = train[train['Rainfall'] < higher_lim]

In [ ]:
higher_lim = train['WindGustSpeed'].quantile(0.995)

train = train[train['WindGustSpeed'] < higher_lim]

## Train Test Split part 2
now we can split target variable and dependent variables:

In [ ]:
X_train = train.drop(['RainTomorrow'], axis=1)
y_train = train['RainTomorrow']

X_test = test.drop(['RainTomorrow'], axis=1)
y_test = test['RainTomorrow']

## Impute missing Data

In [ ]:
# Impute values the naiive approache without considering the locations or other stuff like season

for dataset in [X_train, X_test]:

    colums_containing_nan = dataset.columns[dataset.isnull().any()]
    
    numerical_containing_nan = [col for col in colums_containing_nan if dataset[col].dtypes != 'O']
    categorial_containing_nan = [col for col in colums_containing_nan if dataset[col].dtypes == 'O']

    for col in numerical_containing_nan:
        col_median=X_train[col].median() #always use median from Train data ! Never impute based on Test Data ! we have to assume we dont know it.
        dataset[col] = dataset[col].fillna(col_median) 
        
    for col in categorial_containing_nan:
        col_most_occuring = X_train[col].mode()[0]
        dataset[col] = dataset[col].fillna(col_most_occuring)     
        

## Encode Categorial Variables

using One Hot Encoding...

In [ ]:
# apply One Hot encoding

for col in ["WindGustDir", "WindDir9am", "WindDir3pm", "Season"]:
    encoded_columns = pd.get_dummies(X_train[col], prefix=col, drop_first=True)
    X_train = X_train.join(encoded_columns).drop(col, axis=1)
    
    encoded_columns = pd.get_dummies(X_test[col], prefix=col, drop_first=True)
    X_test = X_test.join(encoded_columns).drop(col, axis=1)

## Scale numerical features

**Note !**
This step is not required for decision trees. Still it is recommended to do. Why?
1. We need it for Feature Selection 
2. If we change the tree to a regression or a NN, we need it.
3. It doesnt do any harm.

In [ ]:
scaler = MinMaxScaler()
cols = X_train.columns

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=cols)
X_test = pd.DataFrame(np.array(X_test), columns=cols)

# Feature Selection


## The univariate way

In [ ]:
selector = SelectKBest(f_classif, k=10)

X_new = selector.fit_transform(X_train, y_train)

selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=X_train.index, 
                                 columns=X_train.columns)

selected_columns = selected_features.columns[selected_features.var() != 0]

X_train[selected_columns].head()

## The multivariate way

In [ ]:
# Regularisation strengh is set by regularization parameter C.
# Note: the lower C, the higher the regularization
logistic = LogisticRegression(C=0.05, penalty="l1", solver='liblinear', max_iter=10000, random_state=7).fit(X_train, y_train)
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(X_train)

selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=X_train.index,
                                 columns=X_train.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]
selected_columns

# Aufgabe 2 Entscheidungsbäume

In [ ]:
model = tree.DecisionTreeClassifier(max_depth=5)

model.fit(X_train[selected_columns], y_train)

In [ ]:
test_predictions = model.predict(X_test[selected_columns]).round().astype(int)
print(accuracy_score(y_test, test_predictions))
mean_absolute_error(y_test, test_predictions)

## Baseline

this is what we got from the beginnnig:
    
0.8435955056179776

0.15640449438202247

## Null accuracy 
Whats the accuracy of a model, which just predicts the most common class.

In [ ]:
y_test.value_counts()

In [ ]:
print(5272/(5272+1403))

## Analysis

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

# Booklet 2 Neuronale Netze Aufgabe 2

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
train_dataset = train_dataset.shuffle(len(X_train)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
test_dataset = test_dataset.shuffle(len(X_test)).batch(len(X_test))

In [ ]:
nn = tf.keras.Sequential([
    keras.layers.Dense(2, activation='relu'),
    keras.layers.Dense(1)
])

In [ ]:
nn.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
nn.fit(train_dataset, 
       validation_steps=1,
       validation_data=test_dataset,
       epochs=10)